In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests 
import json

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from albumentations import (
    Compose, RandomCrop, RandomResizedCrop, HorizontalFlip, VerticalFlip, Resize 
)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import shutil
from PIL import Image



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

BATCH_SIZE = 32
IMG_HEIGHT = 256
IMG_WIDTH = 256
R = 1
base_dir = '/kaggle/working/pokemon/'

In [2]:
____ = set()
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        #____.add(filename.split('.')[-1])
        os.remove(dirname + "/" + filename)
    if dirname in ('/kaggle/working', '/kaggle/working/pokemon'):
        continue
    os.rmdir(dirname)
        
____

set()

In [3]:
!ls

In [4]:
!mkdir pokemon

In [5]:
def is_valid_img(file_path):
    try:
        img = Image.open(file_path)
        return img.format in ("JPG", "JPEG", "PNG")
    except Exception as e:
        print("Error:", e)
        return False

In [6]:
if R != 0:
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            if filename.split('.')[-1].lower() in ('jpeg', 'jpg', 'png',):
                if not dirname.split('/')[-1] in os.listdir(base_dir):
                    os.mkdir(base_dir + dirname.split('/')[-1])
                if is_valid_img(dirname + "/" + filename):
                    shutil.copyfile(dirname + "/" + filename, base_dir + dirname.split('/')[-1] + "/" + filename)


In [7]:
def pokemon_proc_name(pokemon_name):
    if 'mime' in pokemon_name.lower():
        pokemon_name = 'mr-mime'
    return pokemon_name.lower()

def pokemon_get_types(pokemon_name):
    pokemon_info_text = requests.get(f'https://pokeapi.co/api/v2/pokemon/{pokemon_name}').text
    pokemon_info_json = json.loads(pokemon_info_text)
    return tuple([p_type['type']['name'] for p_type in json.loads(pokemon_info_text)['types']])


In [8]:
!ls

pokemon


In [9]:
# Create images df
image_type_check = []
for directory in os.listdir(base_dir):
    if directory.startswith('.') or directory == 'state.db':
        continue
    pokemon_name = pokemon_proc_name(directory)
    pokemon_types = pokemon_get_types(pokemon_name)
    for image in os.listdir(base_dir+directory):
        image_type_check.append([pokemon_name, pokemon_types, f'{directory}/{image}'])

df = pd.DataFrame(image_type_check, columns=['pokemon', 'types', 'filename'])
df = df.reset_index(drop=False)
df.head(3)

,index,pokemon,types,filename
0,0,dragonite,"(dragon, flying)",Dragonite/d4987c9e0b434ca6a1f69633a7c6c1f7.jpg
1,1,dragonite,"(dragon, flying)",Dragonite/00000109.jpg
2,2,dragonite,"(dragon, flying)",Dragonite/00000133.jpg


In [10]:
mlb = MultiLabelBinarizer()
mlb.fit(df['types'].values.tolist())
df['types'] = df['types'].apply(lambda x : (','.join([str(np.where(mlb.classes_ == a)[0][0]) for a in x])))

In [11]:
df.head(3)

,index,pokemon,types,filename
0,0,dragonite,"1,6",Dragonite/d4987c9e0b434ca6a1f69633a7c6c1f7.jpg
1,1,dragonite,"1,6",Dragonite/00000109.jpg
2,2,dragonite,"1,6",Dragonite/00000133.jpg


In [12]:
train_df, test_df = train_test_split(df, test_size=.2)

In [13]:
@tf.function
def multiple_one_hot(cat_tensor, depth):
    one_hot_enc_tensor = tf.zeros(depth, tf.float32)
    
    for cat in cat_tensor:
        one_hot_enc_tensor = tf.add(one_hot_enc_tensor,
                                   tf.one_hot(cat, depth))

    return one_hot_enc_tensor

@tf.function
def proc_types(example):
    splited = tf.strings.split(example['types'], sep=',')
    splited_num = tf.strings.to_number(splited, tf.int32)
    return example["filename"], multiple_one_hot(splited_num, 17)

@tf.function
def decode_image(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=3)
    # Normalize image
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    # resize the image to the desired size
    return img

def resize_val_image(image, label):
    return tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH]), label

@tf.function
def load_image(filename, types):
    # Load image
    image = tf.io.read_file(base_dir + filename)
    image = decode_image(image)
    return image, types

In [14]:
# Define the augmentation policies. Note that they are applied sequentially with some probability p.
transforms = Compose([
                RandomResizedCrop(height=IMG_HEIGHT, width=IMG_WIDTH, always_apply=True),
                HorizontalFlip(),
                VerticalFlip()
        ])

# Apply augmentation policies.
def aug_fn(image):
    data = {"image":image}
    aug_data = transforms(**data)
    aug_img = aug_data["image"]

    return aug_img

In [15]:
@tf.function
def apply_augmentation(image, label):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
    aug_img.set_shape((IMG_HEIGHT, IMG_WIDTH, 3))
    
    return aug_img, label

In [16]:
train_ds = tf.data.Dataset.from_tensor_slices(dict(train_df[['filename','types']]))
test_ds = tf.data.Dataset.from_tensor_slices(dict(train_df[['filename','types']]))
train_ds = (  
    train_ds
    .map(proc_types,num_parallel_calls=tf.data.AUTOTUNE)
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .map(apply_augmentation, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

test_ds = (  
    test_ds
    .map(proc_types,num_parallel_calls=tf.data.AUTOTUNE)
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .map(apply_augmentation, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [17]:
c = 0 
for a in train_ds:
    if c<1:
        print(a)
        c += 1

(<tf.Tensor: shape=(32, 256, 256, 3), dtype=float32, numpy=
array([[[[0.8940931 , 0.80572814, 0.2591365 ],
         [0.8806724 , 0.79367954, 0.25272834],
         [0.8721121 , 0.7899939 , 0.28098917],
         ...,
         [0.8705883 , 0.91498166, 0.8810969 ],
         [0.8705883 , 0.9176471 , 0.87843144],
         [0.8672105 , 0.91034776, 0.8831562 ]],

        [[0.8941177 , 0.8078432 , 0.23767456],
         [0.8924863 , 0.8023027 , 0.2455262 ],
         [0.8918595 , 0.8032642 , 0.26386505],
         ...,
         [0.8705883 , 0.9142383 , 0.8818403 ],
         [0.8705883 , 0.91532636, 0.8807522 ],
         [0.8663833 , 0.91260284, 0.88277143]],

        [[0.8966863 , 0.8129803 , 0.22500767],
         [0.8940871 , 0.8077589 , 0.2354554 ],
         [0.89019614, 0.803868  , 0.23558147],
         ...,
         [0.8705883 , 0.91372555, 0.882353  ],
         [0.8705883 , 0.91372555, 0.882353  ],
         [0.8654156 , 0.91376114, 0.88759154]],

        ...,

        [[0.8836704 , 0.89935666

## Model

In [18]:
class EarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (logs is not None) and (logs.get('accuracy') > 0.90):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training = True

In [22]:
with tf.device('/GPU:0'):

    # Add convolutions and max pooling
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(256, 256, 3)),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(126, activation='relu'),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(17, activation='softmax'),
    ], name='pokemon_model')

    model.summary()

    # Use same settings
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Model: "pokemon_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 254, 254, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 127, 127, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 125, 125, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 60, 60, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 14400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     3,686,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 126)            │        32,382 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 50)             │         6,350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 17)             │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,751,135 (14.31 MB)

 Trainable params: 3,751,135 (14.31 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
print(f'\nMODEL TRAINING:')
with tf.device('/GPU:0'):
    history = model.fit(train_ds,
                        callbacks=[EarlyStopping()],
                        epochs=100)


MODEL TRAINING:
Epoch 1/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 105s 235ms/step - accuracy: 0.1295 - loss: 0.3029
Epoch 2/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 85s 207ms/step - accuracy: 0.2115 - loss: 0.2475
Epoch 3/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 84s 205ms/step - accuracy: 0.2433 - loss: 0.2375
Epoch 4/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 84s 204ms/step - accuracy: 0.2451 - loss: 0.2354
Epoch 5/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 84s 205ms/step - accuracy: 0.2580 - loss: 0.2330
Epoch 6/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 85s 206ms/step - accuracy: 0.2617 - loss: 0.2308
Epoch 7/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 85s 205ms/step - accuracy: 0.2740 - loss: 0.2271
Epoch 8/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 141s 203ms/step - accuracy: 0.2708 - loss: 0.2254
Epoch 9/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 84s 204ms/step - accuracy: 0.2727 - loss: 0.2244
Epoch 10/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 84s 203ms/step - accuracy: 0.2811 - loss: 0.2226
Epoch 11/100
412/412 ━━━━━━━━━━━━━━━━━━━━ 84s 203ms/step - accuracy: 0.2833 - lo

KeyboardInterrupt: 

In [1]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5387826116828616716
 xla_global_id: -1]